We measure the performance of the iron condor method in options trading.  

In [9]:
from polygon import RESTClient

In [10]:
file = open('API_keys.txt','r')
key_text = file.readline()
file.close()
client = RESTClient(api_key=key_text)

In [11]:
# https://github.com/polygon-io/client-python?tab=readme-ov-file
#   1. Get the market price of SPY for time, t
#   2. Estimate +/-3,6 points market calls/puts, m_+3, m_-3, m_+6, m_-6 for expiry, t + 1 (Day)  
#   3. Subtract market price of +/-6 market call/puts
#   4. Add market price of +/-3 market call/puts
options_chain = []
epsilon = 1e-1
for ticker in ["SPY"]:
    for date_ranges in [("2024-05-03", "2024-05-04")]:
        current_day = date_ranges[0]
        next_day = date_ranges[1]
        aggs = client.list_aggs(ticker=ticker, 
                               multiplier=1, 
                               timespan="minute", 
                               from_=current_day, 
                               to=next_day, 
                               limit=50000)
        for agg in aggs:
            market_price = agg.open
            call_options_to_buy = client.list_snapshot_options_chain(
                ticker,
                params={
                    "expiration_date.gte": next_day,
                    "strike_price.gte": market_price + 6 - epsilon,
                    "strike_price.lte": market_price + 6 + epsilon,
                    "contract_type": "call"
                },
            )
            put_options_to_buy = client.list_snapshot_options_chain(
                ticker,
                params={
                    "expiration_date.gte": next_day,
                    "strike_price.gte": market_price - 6 - epsilon,
                    "strike_price.lte": market_price - 6 + epsilon,
                    "contract_type": "put"
                },
            )
            call_options_to_sell = client.list_snapshot_options_chain(
                ticker,
                params={
                    "expiration_date.gte": next_day,
                    "strike_price.gte": market_price + 3 - epsilon,
                    "strike_price.lte": market_price + 3 + epsilon,
                    "contract_type": "call"
                },
            )
            put_options_to_sell = client.list_snapshot_options_chain(
                ticker,
                params={
                    "expiration_date.gte": next_day,
                    "strike_price.gte": market_price - 3 - epsilon,
                    "strike_price.lte": market_price - 3 + epsilon,
                    "contract_type": "put"
                },
            )

ValueError: Invalid header value b'Bearer zoeM1DuFaBOnRcQn_uzVJIpzASpTKfXs\n'